<b>Plan</b>

Step 1: preprocessing sales data and preparing train with predictors *date_block_num, shop_id, item_id, item_category_id* and clipped target

Step 2: concatenate train and test

Step 3: add the following features: *month; nr - number of rows for date_block_num in monthly sales*

Step 4: add “target encoding” derived features with lags 1,2,3,12,13 (l1, l2, l3, l12, l13), and differences between lags 2 and 1 (d2-1), 3 and 2 (d3-2), 13 and 1 (d13-1)

Step 5: add mean_price with lag1: *mean_price=1 if mean item price aggregated by date_block_num/item_id more or equal 4100, mean_price=0 - less than 4100, mean_price=-1 no transactions on an item_id in considered period* )

Step 6: for 3 text variables (shop_name, item_name, item_category_name) make preprocessing (stemming, stop words removal etc.) and tf-idf and/or word-to-vec encoding

Step 7: save train and test to disk as .csv


In [1]:
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [2]:
sales = pd.read_csv('data/sales_train.csv.gz') 
items = pd.read_csv('data/items.csv')
item_categories = pd.read_csv('data/item_categories.csv')
shops = pd.read_csv('data/shops.csv')
test =  pd.read_csv('data/test.csv.gz')

Step 1: preprocessing and preparing train

In [3]:
# drop the row with price 307980.0 as it's outlier and no more rows with item_id 6066 neither in sales no in test
sales.drop(index=sales[sales['item_id']==6066].index, inplace=True)

In [4]:
# change the price of -1 to 2499.0 (see explanation on EDA)
sales.loc[sales['item_price']<0, 'item_price'] = 2499.0

In [5]:
# compose train (monthly_sales): for each month find unique shop_id and item_id, 
# add rows with target=0 that is for each month there are all combinations for found shop_id/item_id

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

index_cols = ['shop_id', 'item_id', 'date_block_num']

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day': 'sum'}).rename(columns = {'item_cnt_day':'target'})

#join aggregated data to the grid
monthly_sales = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

#sort the data
monthly_sales.sort_values(['date_block_num','shop_id','item_id'],inplace=True)


In [6]:
# clip target into [0,20] range
monthly_sales['target'].clip(lower=0, upper=20, inplace=True)

Step 2: concatenate train and test

In [7]:
monthly_sales.head().append(monthly_sales.tail())

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0
10768788,59,22162,33,0.0
10768978,59,22163,33,0.0
10769644,59,22164,33,0.0
10771170,59,22166,33,0.0
10770465,59,22167,33,0.0


In [8]:
len_train = len(monthly_sales)

In [9]:
test.head().append(test.tail())

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648
214199,214199,45,969


In [10]:
len(test)

214200

In [11]:
test = test[['shop_id','item_id']]
test['date_block_num'] = 34

In [12]:
full = pd.concat([monthly_sales, test], ignore_index=True)

In [13]:
full.head().append(full.tail())

,shop_id,item_id,date_block_num,target
0,0,19,0,0.0
1,0,27,0,0.0
2,0,28,0,0.0
3,0,29,0,0.0
4,0,32,0,6.0
11127999,45,18454,34,NaN
11128000,45,16188,34,NaN
11128001,45,15757,34,NaN
11128002,45,19648,34,NaN
11128003,45,969,34,NaN


Step 3: add the following features: *month (extract from date); nr - number of rows for date_block_num in monthly sales*

In [14]:
# add month
m = [i for i in range(1,13)]*2 + [i for i in range(1,12)]
month = pd.DataFrame({'date_block_num':[i for i in range(0,35)], 'month':m})
full = full.merge(month, how='left', on='date_block_num')

In [15]:
# add nrow_block
n = full.groupby('date_block_num', as_index=False).size().rename(columns={'size':'nrow_block'})
full = full.merge(n, how='left', on='date_block_num')

In [16]:
full.head()

,shop_id,item_id,date_block_num,target,month,nrow_block
0,0,19,0,0.0,1,365175
1,0,27,0,0.0,1,365175
2,0,28,0,0.0,1,365175
3,0,29,0,0.0,1,365175
4,0,32,0,6.0,1,365175


Step 4: add “target encoding” derived features from the group l1, l2, l3, l12, l13, d2-1, d3-2, d13-1 <br>
First we add “target encoding” features and then compose derived features
<p>“target encoding” features:
<ul>
<li>tg_sum: sum of target at date_block_num level
<li>tg_mean: mean of target at date_block_num level
<li>tg_shop: mean of target by shop aggregated at date_block_num level
<li>tg_item: mean of target by item aggregated at date_block_num level
<li>tg_cat: mean of target by category aggregated at date_block_num level
<li>tg: target    
</ul>
Target encording (6) + derived features (48) = 54 features

In [17]:
# add item_category_id 
full = full.merge(items.drop('item_name', axis=1), how='left', on='item_id')

In [18]:
# tg_sum, tg_mean
ms = full.groupby('date_block_num')['target'].agg(['sum', 'mean']).rename(columns={'sum':'tg_sum', 'mean':'tg_mean'})
ms.reset_index(level=0, inplace=True)
full = full.merge(ms, how='left', on='date_block_num')


In [19]:
# tg_shop, tg_item, tg_cat
col = ['shop', 'item', 'item_category']
for i in col:
    gcol = i + '_id'
    agg_name = 'tg_' + i
    ms = full.groupby(['date_block_num', gcol], as_index=False)['target'].agg( 'mean').rename(columns={'target':agg_name})
    full = full.merge(ms, how='left', on=['date_block_num', gcol]).fillna(0)
full.rename(columns={'tg_item_category':'tg_cat'}, inplace=True)


In [20]:
# add tg column as copy target
full['tg'] = full['target']

In [21]:
# add lags 1,2,3,12,13 for all target encoded vars (var name starts with 'tg')
lag_var = full.columns[full.columns.str.startswith('tg')]
lags = [1,2,3,12,13]
for lag in lags:
    lag_df = full.copy()
    lag_df['date_block_num'] += lag
    lag_df = lag_df[['date_block_num', 'shop_id', 'item_id'] + list(lag_var)]
    lag_df.columns = ['date_block_num', 'shop_id', 'item_id'] + \
    [lv +'_lag'+str(lag) for lv in lag_var]
    full = full.merge(lag_df,on=['date_block_num','shop_id','item_id'] ,how='left').fillna(0)
    

In [22]:
for i in lag_var:
    for (j1,j2) in [(2,1),(3,2),(13,1)]:
        col1 = i+'_lag'+str(j1)
        col2 = i+'_lag'+str(j2)
        newcol = i+'_d'+str(j1)+'-'+str(j2)
        full[newcol] = full[col1] - full[col2]
        

Step 5: add mean_price with lag1: *mean_price=1 if mean item price aggregated by date_block_num/item_id more or equal 4100, mean_price=0 - less than 4100, mean_price=-1 no transactions on an item_id in considered period* )

In [23]:
# add mean_price
price = sales.groupby(['date_block_num', 'item_id'], as_index=False).agg({'item_price':'mean'})\
.rename(columns={'item_price':'mean_price_lag1'})
price['mean_price_lag1'] = np.where(price['mean_price_lag1'] >= 4100, 1, 0)
price['date_block_num'] += 1
full = full.merge(price, how='left', on=['date_block_num', 'item_id']).fillna(-1)

Step 6: for 3 text variables (shop_name, item_name, item_category_name) make preprocessing (stemming, stop words removal etc.) and tf-idf and/or word-to-vec encoding

Step 7: save train and test to disk as .csv

In [28]:
monthly_sales = full[:len_train]
test = full[len_train:].drop('target', axis=1)

In [29]:
# len(test)

In [30]:
%%time
# save monthly_sales to disk
monthly_sales.to_csv('data/train_numfeatures.csv.gz', index=False, compression='gzip')
test.to_csv('data/test_numfeatures.csv.gz', index=False, compression='gzip')


CPU times: user 10min 40s, sys: 4.39 s, total: 10min 45s
Wall time: 10min 46s
